In [1]:
from whispr import *

In [3]:
%%capture

'''
To add:


* update volumes on source plate at end (requires better keeping track of volume used)

* better commenting, use of functions / improving of readability

'''



In [4]:
"""
For user to change according to run
"""

#CSV file with source plate info. Columns are label, well, concentration, volume
source_plate = 'source_plates/ECHO_source_plate.xlsx'
source_plate_df = pd.read_excel(source_plate, index_col = 0)

filename = '211021-titrations-test1.csv'
# CSV file with output plate layout: columns labeled 1-12 and rows labeled A-H for 96 well plate
output_layout = 'plate_layouts/'+filename 

#CSV file with each reaction as rows and the volume of each input (columns) added to reach final reaction volume
mixing_table = 'mixing_tables/'+filename 
mixing_table_df = pd.read_csv(mixing_table, index_col = 0, dtype = str).fillna(0)

source_plate_type = '384PP_AQ_BP' 

checkInputs(source_plate_df, mixing_table,source_plate_type)

vol_table_df = generateVolumeTable(mixing_table_df, source_plate_df)

output_df = writeProtocol(source_plate_type, vol_table_df, source_plate_df, output_layout,source_plate_df)
output_df.to_csv('protocols/'+filename,index = False)

AttributeError: 'numpy.float64' object has no attribute 'sort_values'

In [10]:
mixing_table_df

In [4]:
output_df

,Source Plate Name,Source Plate Type,Source Well,Destination Plate Name,Destination Well,Transfer Volume
0,Source[1],384PP_AQ_BP,A6,Destination[1],A1,675.0
1,Source[1],384PP_AQ_BP,A1,Destination[1],A1,1825.0
2,Source[1],384PP_AQ_BP,A6,Destination[1],B1,675.0
3,Source[1],384PP_AQ_BP,A1,Destination[1],B1,1825.0
4,Source[1],384PP_AQ_BP,A6,Destination[1],C1,675.0
5,Source[1],384PP_AQ_BP,A1,Destination[1],C1,1825.0
6,Source[1],384PP_AQ_BP,A6,Destination[1],A2,75.0
7,Source[1],384PP_AQ_BP,A1,Destination[1],A2,2425.0
8,Source[1],384PP_AQ_BP,A6,Destination[1],B2,75.0
9,Source[1],384PP_AQ_BP,A1,Destination[1],B2,2425.0
